In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import lyricsgenius
from textblob import TextBlob
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor
import csv
import os
from dotenv import load_dotenv

In [ ]:
# Load environment variables from .env
load_dotenv()

# Set up your Spotify API credentials
SPOTIPY_CLIENT_ID = os.getenv('SPOTIPY_CLIENT_ID')
SPOTIPY_CLIENT_SECRET = os.getenv('SPOTIPY_CLIENT_SECRET')

# Set up your Genius API credentials
GENIUS_ACCESS_TOKEN = os.getenv('GENIUS_ACCESS_TOKEN')

In [ ]:
# Set up Spotify API authentication
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=SPOTIPY_CLIENT_ID, client_secret=SPOTIPY_CLIENT_SECRET))

# Set up Genius API authentication
genius = lyricsgenius.Genius(GENIUS_ACCESS_TOKEN)

In [ ]:
def get_lyrics(track_name, artist_name):
    # Search for lyrics using Genius API
    song = genius.search_song(title=track_name, artist=artist_name)
    if song:
        return song.lyrics
    else:
        return None

def analyze_sentiment(lyrics):
    # Analyze sentiment using TextBlob
    blob = TextBlob(lyrics)
    sentiment = blob.sentiment.polarity
    return sentiment

def analyze_track(track):
    try:
        track_name = track['track']['name']
        artist_name = track['track']['artists'][0]['name']

        # Get lyrics for the current track
        lyrics = get_lyrics(track_name, artist_name)

        if lyrics:
            # Analyze sentiment
            sentiment = analyze_sentiment(lyrics)

            return {
                'track_name': track_name,
                'artist_name': artist_name,
                'sentiment': sentiment
            }
    except Exception as e:
        print(f"Something went wrong: {e}")
        return None

def analyze_playlist(playlist_uri):
    # Get playlist details from Spotify
    playlist = sp.playlist_tracks(playlist_uri)
    
    # List to store results
    results = []

    with ThreadPoolExecutor() as executor:
        # Use tqdm to create a progress bar
        for result in tqdm(executor.map(analyze_track, playlist['items']), total=len(playlist['items']), desc='Analyzing'):
            if result:
                results.append(result)

    # Sort results based on sentiment
    sorted_results = sorted(results, key=lambda x: x['sentiment'], reverse=True)

    # Print sorted results
    for result in sorted_results:
        print(f"Track: {result['track_name']} - Artist: {result['artist_name']}")
        print(f"Sentiment: {result['sentiment']}")
        print("-" * 30)

    return sorted_results

In [ ]:
# run for a specific playlist
playlist_uri = 'https://open.spotify.com/playlist/1hblPa5R3QgJGWoazGIbWG?si=e9214b8bbe374cfa'
results = analyze_playlist(playlist_uri)

In [ ]:
# Print and write sorted results to CSV
folder_name = 'sentiment_analysis'
os.makedirs(folder_name, exist_ok=True)

playlist_name = sp.playlist(playlist_uri)['name']
csv_filename = os.path.join(folder_name, f"{playlist_name}_sentiment.csv")

with open(csv_filename, 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['track_name', 'artist_name', 'sentiment']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for result in results:
        writer.writerow(result)

print(f"Results written to {csv_filename}")